### Data Acquisition

In [1]:
# all modules

import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
engine = create_engine('sqlite:///../data/raw/raw_data_project_m1.db')

In [6]:
query="""
SELECT country_info.country_code,
career_info.normalized_job_code,
personal_info.age
FROM poll_info 
JOIN personal_info 
ON poll_info.uuid = personal_info.uuid 
JOIN career_info 
ON poll_info.uuid = career_info.uuid 
JOIN country_info 
ON poll_info.uuid = country_info.uuid
"""

In [7]:
df_data_raw = pd.read_sql_query(query, engine)

In [5]:
df_data_raw.isnull().any()

country_code           False
normalized_job_code     True
age                    False
dtype: bool

In [14]:
df_data_raw.dropna(inplace=True)
df_data_raw.reset_index(inplace=True)
df_data_raw.drop('index', axis=1,inplace=True)
df_data_raw

,country_code,normalized_job_code,age
0,AT,861a9b9151e11362eb3c77ca914172d0,57 years old
1,AT,049a3f3a2b5f85cb2971ba77ad66e10c,45 years old
2,AT,f4b2fb1aa40f661488e2782b6d57ad2f,41 years old
3,AT,27af8700f5577cec835acee2cb90a2ff,1990
4,AT,c1b670eba9ccb65e7c99f7da116d5b9c,47 years old
...,...,...,...
5697,SK,41b2978a2c07c5f477e60f26372d6c85,2000
5698,SK,847165cfda6b1dc82ae22b967da8af2f,37 years old
5699,SK,a4d5b8b38f9513825d0d94a981ebe962,53 years old
5700,SK,775190277a849cba701b306a7b374c0a,47 years old


In [31]:
df_data_raw.rename({'country':'Country'}, axis=1, inplace=True)
df_data_raw

,Country,normalized_job_code,age
0,AT,None,61 years old
1,AT,861a9b9151e11362eb3c77ca914172d0,57 years old
2,AT,None,32 years old
3,AT,049a3f3a2b5f85cb2971ba77ad66e10c,45 years old
4,AT,f4b2fb1aa40f661488e2782b6d57ad2f,41 years old
...,...,...,...
9644,SK,847165cfda6b1dc82ae22b967da8af2f,37 years old
9645,SK,a4d5b8b38f9513825d0d94a981ebe962,53 years old
9646,SK,None,1992
9647,SK,775190277a849cba701b306a7b374c0a,47 years old


In [52]:

country='ES'
df_choice = df_data_raw[df_data_raw['country_code']==f'{country}'].reset_index()

df_choice

,index,country_code,normalized_job_code,age
0,2212,ES,7028f15ad274cb4d9505101b9c1306b1,59
1,2213,ES,a12a16280e22220f9354333f6fe1b22a,41
2,2214,ES,e605336f5a43949cb6b18054fd1937f1,52
3,2215,ES,0facaffe51cc1998a5fbdbe5fd18fd98,45
4,2216,ES,a12a16280e22220f9354333f6fe1b22a,50
...,...,...,...,...
1000,3212,ES,None,28
1001,3213,ES,None,25
1002,3214,ES,dac2e921c1f887d9efc4c06fa33c1397,57
1003,3215,ES,None,52


In [31]:
df_data_raw.loc[:,:]

,country_code,normalized_job_code,age
0,AT,None,61 years old
1,AT,861a9b9151e11362eb3c77ca914172d0,57 years old
2,AT,None,32 years old
3,AT,049a3f3a2b5f85cb2971ba77ad66e10c,45 years old
4,AT,f4b2fb1aa40f661488e2782b6d57ad2f,41 years old
...,...,...,...
9644,SK,847165cfda6b1dc82ae22b967da8af2f,37 years old
9645,SK,a4d5b8b38f9513825d0d94a981ebe962,53 years old
9646,SK,None,1992
9647,SK,775190277a849cba701b306a7b374c0a,47 years old


In [8]:
#limpiar años
df_data_raw['age']=df_data_raw['age'].str.replace(r'[a-zA-Z]','')

In [9]:
df_data_raw['age']=df_data_raw['age'].astype(int)

count=0
for i in df_data_raw['age']:
    if i>1000:
        df_data_raw.loc[count,'age']=2016-i
    count+=1

df_data_raw

,country_code,normalized_job_code,age
0,AT,None,61
1,AT,861a9b9151e11362eb3c77ca914172d0,57
2,AT,None,32
3,AT,049a3f3a2b5f85cb2971ba77ad66e10c,45
4,AT,f4b2fb1aa40f661488e2782b6d57ad2f,41
...,...,...,...
9644,SK,847165cfda6b1dc82ae22b967da8af2f,37
9645,SK,a4d5b8b38f9513825d0d94a981ebe962,53
9646,SK,None,24
9647,SK,775190277a849cba701b306a7b374c0a,47


In [10]:
jobs_id=df_data_raw['normalized_job_code'].unique()
len(jobs_id)

157

In [11]:
import requests

json_complete=[]

for i in jobs_id:
    response = requests.get(f'http://api.dataatwork.org/v1/jobs/{i}')
    json_data = response.json()
    json_complete.append(json_data)


In [61]:
json_complete[0:5]

[{'error': {'code': 404, 'message': 'Cannot find job with id None'}},
 {'uuid': '861a9b9151e11362eb3c77ca914172d0',
  'title': 'Automatic Data Processing Planner',
  'normalized_job_title': 'automatic data processing planner',
  'parent_uuid': '0148f61d4227497728ce33490843d056'},
 {'uuid': '049a3f3a2b5f85cb2971ba77ad66e10c',
  'title': 'Data Coordinator',
  'normalized_job_title': 'data coordinator',
  'parent_uuid': '0b9dd32a367f4562ec77b993053d1910'},
 {'uuid': 'f4b2fb1aa40f661488e2782b6d57ad2f',
  'title': 'Database Developer',
  'normalized_job_title': 'database developer',
  'parent_uuid': 'b90ca4df5690002377a7b0f1f3d40781'},
 {'uuid': '27af8700f5577cec835acee2cb90a2ff',
  'title': 'Data Entry Specialist',
  'normalized_job_title': 'data entry specialist',
  'parent_uuid': 'bf88358c56bb6cbe7eabec38ae333d19'}]

In [54]:
json_complete[1]

{'uuid': '861a9b9151e11362eb3c77ca914172d0',
 'title': 'Automatic Data Processing Planner',
 'normalized_job_title': 'automatic data processing planner',
 'parent_uuid': '0148f61d4227497728ce33490843d056'}

In [12]:
count=0
for code in df_data_raw['normalized_job_code']:
    for job in json_complete:
        if (job.get('uuid')==code) and (code!=None):
            df_data_raw.loc[count,'normalized_job_code']=job.get('title')
    count+=1
    

In [15]:
df_data_raw[0:70]

,country_code,normalized_job_code,age
0,AT,None,61
1,AT,Automatic Data Processing Planner,57
2,AT,None,32
3,AT,Data Coordinator,45
4,AT,Database Developer,41
...,...,...,...
65,AT,None,47
66,AT,None,25
67,AT,Computer or Data Processing Systems Consultant,65
68,AT,Data Coordinator,46


In [16]:
count=0
for code in df_data_raw['normalized_job_code']:
    if code==None:
        df_data_raw.loc[count,'normalized_job_code']='Jobless'
    count+=1

df_data_raw

,country_code,normalized_job_code,age
0,AT,Jobless,61
1,AT,Automatic Data Processing Planner,57
2,AT,Jobless,32
3,AT,Data Coordinator,45
4,AT,Database Developer,41
...,...,...,...
9644,SK,Data Warehouse Developer,37
9645,SK,Database Manager,53
9646,SK,Jobless,24
9647,SK,Data Officer,47


In [17]:
import requests
from bs4 import BeautifulSoup
url = 'https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes'
html = requests.get(url).text
html[:5000]
soup = BeautifulSoup(html, 'lxml')
table = soup.find_all('td')


countries_raw=[]
for i in table:
    countries_raw.append(i.text)

countries_raw[0:5]

['Belgium\n', '(BE)\n', 'Greece\n', '(EL)\n', 'Lithuania\n']

In [18]:
#clean countries
import re

countries=[]
for i in countries_raw:
    a=re.sub('\n','',i)
    b=re.sub('^ ','',a)
    try:
        if b[0]=='(':
            b=b[1:3]
    except:
        continue
        
    countries.append(b)

countries[:58]


['Belgium',
 'BE',
 'Greece',
 'EL',
 'Lithuania',
 'LT',
 'Portugal',
 'PT',
 'Bulgaria',
 'BG',
 'Spain',
 'ES',
 'Luxembourg',
 'LU',
 'Romania',
 'RO',
 'Czechia',
 'CZ',
 'France',
 'FR',
 'Hungary',
 'HU',
 'Slovenia',
 'SI',
 'Denmark',
 'DK',
 'Croatia',
 'HR',
 'Malta',
 'MT',
 'Slovakia',
 'SK',
 'Germany',
 'DE',
 'Italy',
 'IT',
 'Netherlands',
 'NL',
 'Finland',
 'FI',
 'Estonia',
 'EE',
 'Cyprus',
 'CY',
 'Austria',
 'AT',
 'Sweden',
 'SE',
 'Ireland',
 'IE',
 'Latvia',
 'LV',
 'Poland',
 'PL',
 'United Kingdom',
 'UK',
 'Iceland',
 'IS']

In [19]:
countries_library={}
for i in range(1,57,2):
    countries_library[f'{countries[i]}']=countries[i-1]

countries_library


{'BE': 'Belgium',
 'EL': 'Greece',
 'LT': 'Lithuania',
 'PT': 'Portugal',
 'BG': 'Bulgaria',
 'ES': 'Spain',
 'LU': 'Luxembourg',
 'RO': 'Romania',
 'CZ': 'Czechia',
 'FR': 'France',
 'HU': 'Hungary',
 'SI': 'Slovenia',
 'DK': 'Denmark',
 'HR': 'Croatia',
 'MT': 'Malta',
 'SK': 'Slovakia',
 'DE': 'Germany',
 'IT': 'Italy',
 'NL': 'Netherlands',
 'FI': 'Finland',
 'EE': 'Estonia',
 'CY': 'Cyprus',
 'AT': 'Austria',
 'SE': 'Sweden',
 'IE': 'Ireland',
 'LV': 'Latvia',
 'PL': 'Poland',
 'UK': 'United Kingdom'}

In [20]:
list(countries_library.values())

['Belgium',
 'Greece',
 'Lithuania',
 'Portugal',
 'Bulgaria',
 'Spain',
 'Luxembourg',
 'Romania',
 'Czechia',
 'France',
 'Hungary',
 'Slovenia',
 'Denmark',
 'Croatia',
 'Malta',
 'Slovakia',
 'Germany',
 'Italy',
 'Netherlands',
 'Finland',
 'Estonia',
 'Cyprus',
 'Austria',
 'Sweden',
 'Ireland',
 'Latvia',
 'Poland',
 'United Kingdom']

In [39]:
country = 'spAin'

country=country.lower().capitalize()
if country in list(countries_library.values()):
    print('ok')


ok


In [21]:
count=0
for i in df_data_raw['country_code']:
    if i=='GB':
        df_data_raw.loc[count,'country_code']='UK'
    elif i=='GR':
        df_data_raw.loc[count,'country_code']='EL'
    count+=1
    
df_data_raw

,country_code,normalized_job_code,age
0,AT,Jobless,61
1,AT,Automatic Data Processing Planner,57
2,AT,Jobless,32
3,AT,Data Coordinator,45
4,AT,Database Developer,41
...,...,...,...
9644,SK,Data Warehouse Developer,37
9645,SK,Database Manager,53
9646,SK,Jobless,24
9647,SK,Data Officer,47


In [22]:
count=0
for country in df_data_raw['country_code']:
    df_data_raw.loc[count,'country_code']=countries_library[country]
    count+=1


In [23]:
df_data_raw.head(10)

,country_code,normalized_job_code,age
0,Austria,Jobless,61
1,Austria,Automatic Data Processing Planner,57
2,Austria,Jobless,32
3,Austria,Data Coordinator,45
4,Austria,Database Developer,41
5,Austria,Data Entry Specialist,26
6,Austria,Jobless,26
7,Austria,Database Architect,47
8,Austria,Geographic Information Systems Database Admini...,34
9,Austria,Jobless,22


In [24]:
count=0
for i in df_data_raw['age']:
    if i<26:
        df_data_raw.loc[count,'age_group']= '14_25'
    elif i>39:
        df_data_raw.loc[count,'age_group']= '40_65'
    else:
        df_data_raw.loc[count,'age_group']= '26_39'
    count+=1
df_data_raw

,country_code,normalized_job_code,age,age_group
0,Austria,Jobless,61,40_65
1,Austria,Automatic Data Processing Planner,57,40_65
2,Austria,Jobless,32,26_39
3,Austria,Data Coordinator,45,40_65
4,Austria,Database Developer,41,40_65
...,...,...,...,...
9644,Slovakia,Data Warehouse Developer,37,26_39
9645,Slovakia,Database Manager,53,40_65
9646,Slovakia,Jobless,24,14_25
9647,Slovakia,Data Officer,47,40_65


In [25]:
df_data_raw

,country_code,normalized_job_code,age,age_group
0,Austria,Jobless,61,40_65
1,Austria,Automatic Data Processing Planner,57,40_65
2,Austria,Jobless,32,26_39
3,Austria,Data Coordinator,45,40_65
4,Austria,Database Developer,41,40_65
...,...,...,...,...
9644,Slovakia,Data Warehouse Developer,37,26_39
9645,Slovakia,Database Manager,53,40_65
9646,Slovakia,Jobless,24,14_25
9647,Slovakia,Data Officer,47,40_65


In [26]:
df_hola = df_data_raw[df_data_raw['country_code']=='Spain']
# df_hola=df_hola.drop(['index'],axis=1)
df_hola.reset_index(inplace=True)
df_hola

,index,country_code,normalized_job_code,age,age_group
0,2212,Spain,Clinical Data Management Manager (CDM Manager),59,40_65
1,2213,Spain,Geographic Information Systems Data Administra...,41,40_65
2,2214,Spain,Database Security Expert,52,40_65
3,2215,Spain,Weight in Motion Field Data Collection Technician,45,40_65
4,2216,Spain,Geographic Information Systems Data Administra...,50,40_65
...,...,...,...,...,...
1000,3212,Spain,Jobless,28,26_39
1001,3213,Spain,Jobless,25,14_25
1002,3214,Spain,Automatic Data Processing Customer Liaison (AD...,57,40_65
1003,3215,Spain,Jobless,52,40_65


In [27]:
df_3=df_data_raw.groupby(['country_code','normalized_job_code','age_group']).count()
print(df_3.columns)

Index(['age'], dtype='object')


In [30]:
df_3.columns=['Quantity']
df_3.reset_index(inplace=True)
df_3

ValueError: Length mismatch: Expected axis has 4 elements, new values have 1 elements

In [31]:
df_4=(df_3[['Quantity']]/df_3[['Quantity']].sum())*100
df_4=df_4.round(2)
df_4

,Quantity
0,0.01
1,0.01
2,0.02
3,0.01
4,0.01
...,...
2949,0.01
2950,0.03
2951,0.02
2952,0.03


In [80]:
# df_1=df_data_raw.groupby(by=['country_code','normalized_job_code','age_group']).agg({'age':'count'})
# df_1.columns=['Count']
# df_1=df_1.reset_index()
# df_1

In [32]:
df_4.columns=['Percentage']

In [33]:
df_3

,country_code,normalized_job_code,age_group,Quantity
0,Austria,Automatic Data Processing Customer Liaison (AD...,26_39,1
1,Austria,Automatic Data Processing Customer Liaison (AD...,40_65,1
2,Austria,Automatic Data Processing Planner,40_65,2
3,Austria,Automatic Data Processing Systems Security Spe...,26_39,1
4,Austria,Automatic Data Processing Systems Security Spe...,40_65,1
...,...,...,...,...
2949,United Kingdom,SCADA Technician (Supervisory Control and Data...,26_39,1
2950,United Kingdom,SQL Database Administrator,14_25,3
2951,United Kingdom,SQL Database Administrator,26_39,2
2952,United Kingdom,SQL Database Administrator,40_65,3


In [54]:
df_3[['Quantity']].sum()

Quantity    9649
dtype: int64

In [82]:
df_4

,Percentage
0,0.01
1,0.01
2,0.02
3,0.01
4,0.01
...,...
2949,0.01
2950,0.03
2951,0.02
2952,0.03


In [37]:
df_total=pd.merge(df_3, df_4, left_index=True, right_index=True)

df_total[0:70]

,country_code,normalized_job_code,age_group,Quantity,Percentage
0,Austria,Automatic Data Processing Customer Liaison (AD...,26_39,1,0.01
1,Austria,Automatic Data Processing Customer Liaison (AD...,40_65,1,0.01
2,Austria,Automatic Data Processing Planner,40_65,2,0.02
3,Austria,Automatic Data Processing Systems Security Spe...,26_39,1,0.01
4,Austria,Automatic Data Processing Systems Security Spe...,40_65,1,0.01
...,...,...,...,...,...
65,Austria,Jobless,40_65,11,0.11
66,Austria,Maintenance Data Analyst,40_65,1,0.01
67,Austria,Naval Tactical Data System--Combat Information...,14_25,1,0.01
68,Austria,Naval Tactical Data System--Combat Information...,26_39,1,0.01
